In [1]:
import pandas as pd

In [2]:
file = "D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\train_E6oV3lV.csv"
test = 'D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\test_tweets_anuFYb8.csv'
tweets = pd.read_csv(file)
test_tweets=pd.read_csv(test)

In [3]:
tweets1=tweets[tweets.label==1]
print(tweets1.count())

id       2242
label    2242
tweet    2242
dtype: int64


In [4]:
tweets=tweets.append(tweets1)
tweets=tweets.append(tweets1)
tweets=tweets.append(tweets1)

In [5]:
label=tweets.label

In [6]:
tweets[tweets.label==1].count()

id       8968
label    8968
tweet    8968
dtype: int64

In [7]:
tweets=tweets.append(test_tweets)

C:\Amrendra\Anaconda\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [8]:
tweets.groupby(tweets.label).tweet.count()

label
0.0    29720
1.0     8968
Name: tweet, dtype: int64

In [9]:
cleaned_tweet = tweets.tweet.str.replace("[^a-zA-Z]", " ")

In [10]:
cleaned_tweet=cleaned_tweet.apply(lambda x :" ".join(w.lower() for w in x.split() if  w != 'user' ))

In [11]:
from nltk.stem.porter import PorterStemmer
#from nltk.stem import WordNetLemmatizer

In [12]:
stem=PorterStemmer()

In [13]:
cleaned_tweet = cleaned_tweet.apply(lambda x : " ".join( stem.stem(w) for w in x.split()))

In [14]:
cleaned_tweet = cleaned_tweet.apply(lambda x : " ".join( w for w in x.split()))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfidf = TfidfVectorizer(max_features=9300,stop_words='english')

In [17]:
bow=tfidf.fit_transform(cleaned_tweet)

In [80]:
train_bow = bow[:38688,:]
test_bow = bow[38688:,:]

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_test,y_train, y_test = train_test_split(train_bow,label,test_size=0.2,random_state=3)

In [21]:
from sklearn import svm
model_svm= svm.SVC(kernel='rbf', C=1,gamma=1)
model_svm.fit(x_train,y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
pred_svm=model_svm.predict(x_test)

In [23]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [24]:
metrics.accuracy_score(y_test,pred_svm)

0.990695270095632

In [25]:
test_pred_svm= model_svm.predict(test_bow)

In [26]:
from sklearn.naive_bayes import MultinomialNB
model_NB= MultinomialNB(0.25)
model_NB.fit(x_train,y_train)
pred_NB=model_NB.predict(x_test)
from sklearn import metrics
from sklearn.metrics import confusion_matrix
metrics.accuracy_score(y_test,pred_NB)

0.9579994830705608

In [27]:
test_pred_NB= model_NB.predict(test_bow)

In [28]:
from sklearn.linear_model import LogisticRegression
model_LR= LogisticRegression()
model_LR.fit(x_train,y_train)
test_pred_LR= model_LR.predict(test_bow)

C:\Amrendra\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [277]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
model_xgb = XGBClassifier( nthread=4, n_estimators=200, learning_rate=0.2,
       colsample_bytree=0.85,
        max_depth=50,
        reg_lambda=3)
model_xgb.fit(x_train,y_train)
pred_xgb=model_xgb.predict(x_test)

from sklearn import metrics
from sklearn.metrics import confusion_matrix

metrics.accuracy_score(y_test,pred_xgb)

0.9720858102868959

In [278]:
test_pred_xgb= model_xgb.predict(test_bow)

In [82]:
import os
output_dir = 'model_tweet_output1/dense'
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [85]:
x_train,x_test,y_train, y_test = train_test_split(train_bow,label,test_size=0.2,random_state=3)

model_mlp_tfidf = Sequential()
model_mlp_tfidf.add(Dense(3000,activation='relu', input_shape=(9300,)))
model_mlp_tfidf.add(Dropout(0.2))
model_mlp_tfidf.add(Dense(1,activation='sigmoid'))
model_mlp_tfidf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_mlp_tfidf.fit(x_train,y_train,epochs=3,verbose=1, validation_data=(x_test,y_test),callbacks=[modelcheckpoint])

Train on 30950 samples, validate on 7738 samples
Epoch 1/3
30950/30950 [==============================] - 533s 17ms/step - loss: 0.1629 - acc: 0.9346 - val_loss: 0.0788 - val_acc: 0.9707
Epoch 2/3
30950/30950 [==============================] - 513s 17ms/step - loss: 0.0307 - acc: 0.9897 - val_loss: 0.0486 - val_acc: 0.9854
Epoch 3/3
30950/30950 [==============================] - 519s 17ms/step - loss: 0.0092 - acc: 0.9976 - val_loss: 0.0454 - val_acc: 0.9866


In [109]:
model_mlp_tfidf.load_weights(output_dir+'/weights.03.hdf5')
test_pred_mlp_tfidf=model_mlp_tfidf.predict_classes(test_bow)

In [43]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_tweet)
sequences = tokenizer.texts_to_sequences(cleaned_tweet)

from keras.preprocessing.sequence import pad_sequences
word_index=pad_sequences(sequences, maxlen=40, padding = 'pre', truncating='pre')

train = word_index[:38688,:]
test = word_index[38688:,:]

from sklearn.model_selection import train_test_split

x_train,x_test,y_train, y_test = train_test_split(train,label,test_size=0.2,random_state=3, stratify=label)

In [45]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.layers import Conv1D, SpatialDropout1D, GlobalMaxPooling1D
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import pandas as pd
import matplotlib.pyplot as ply
%matplotlib inline

output_dir = 'model_tweet_output/LSTM'
epochs = 5
batch_size = 64
n_dim = 64
n_unique_words = 50000
n_words_to_skip = 5
max_review_length = 40
pad_type = trunc_type = 'pre'

model_lstm = Sequential()
model_lstm.add(Embedding(n_unique_words,n_dim, input_length=max_review_length))
model_lstm.add(LSTM(30,return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(30))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model_lstm.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test,y_test), callbacks=[modelcheckpoint])

Train on 30950 samples, validate on 7738 samples
Epoch 1/5
30950/30950 [==============================] - 57s 2ms/step - loss: 0.2453 - acc: 0.9015 - val_loss: 0.1217 - val_acc: 0.9563
Epoch 2/5
30950/30950 [==============================] - 57s 2ms/step - loss: 0.0662 - acc: 0.9783 - val_loss: 0.0779 - val_acc: 0.9711
Epoch 3/5
30950/30950 [==============================] - 61s 2ms/step - loss: 0.0280 - acc: 0.9904 - val_loss: 0.0627 - val_acc: 0.9778
Epoch 4/5
30950/30950 [==============================] - 58s 2ms/step - loss: 0.0149 - acc: 0.9951 - val_loss: 0.1000 - val_acc: 0.9740
Epoch 5/5
30950/30950 [==============================] - 59s 2ms/step - loss: 0.0083 - acc: 0.9974 - val_loss: 0.0812 - val_acc: 0.9796


In [60]:
model_lstm.load_weights(output_dir+'/weights.05.hdf5')
test_pred_lstm = model_lstm.predict_classes(test)

In [68]:
output_dir = 'model_tweet_output/cnn'
epochs = 5
batch_size = 128
n_dim = 128
n_unique_words = 50000
n_words_to_skip = 5
max_review_length = 40
pad_type = trunc_type = 'pre'
drop_embed = 0.2

n_dense = 256
dropout = 0.2

n_conv = 256
k_conv = 3

In [69]:
model_cnn = Sequential()
model_cnn.add(Embedding(n_unique_words,n_dim, input_length=max_review_length))
model_cnn.add(SpatialDropout1D(drop_embed))
model_cnn.add(Conv1D(n_conv, k_conv, activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(n_dense,activation='relu'))
model_cnn.add(Dropout(dropout))
model_cnn.add(Dense(1, activation='sigmoid'))

model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model_cnn.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test,y_test), callbacks=[modelcheckpoint])

Train on 30950 samples, validate on 7738 samples
Epoch 1/5
30950/30950 [==============================] - 526s 17ms/step - loss: 0.2578 - acc: 0.8913 - val_loss: 0.1047 - val_acc: 0.9592
Epoch 2/5
30950/30950 [==============================] - 511s 17ms/step - loss: 0.0534 - acc: 0.9818 - val_loss: 0.0550 - val_acc: 0.9810
Epoch 3/5
30950/30950 [==============================] - 517s 17ms/step - loss: 0.0143 - acc: 0.9955 - val_loss: 0.0670 - val_acc: 0.9793
Epoch 4/5
30950/30950 [==============================] - 513s 17ms/step - loss: 0.0046 - acc: 0.9986 - val_loss: 0.0652 - val_acc: 0.9846
Epoch 5/5
30950/30950 [==============================] - 519s 17ms/step - loss: 0.0022 - acc: 0.9995 - val_loss: 0.0716 - val_acc: 0.9829


In [74]:
model_cnn.load_weights(output_dir+'/weights.04.hdf5')
test_pred_cnn = model_cnn.predict_classes(test)

In [75]:
import numpy as np

In [110]:
test_pred_NB = pd.DataFrame(test_pred_NB)
test_pred_svm = pd.DataFrame(test_pred_svm)
test_pred_LR = pd.DataFrame(test_pred_LR)
test_pred_xgb = pd.DataFrame(test_pred_xgb)
test_pred_lstm = pd.DataFrame(test_pred_lstm)
test_pred_cnn = pd.DataFrame(test_pred_cnn)
test_pred_mlp_tfidf = pd.DataFrame(test_pred_mlp_tfidf)

In [111]:
output = pd.concat([test_pred_NB,test_pred_svm,test_pred_LR,test_pred_xgb,test_pred_lstm,test_pred_cnn, test_pred_mlp_tfidf],axis=1)

In [112]:
output.mode(axis=1).to_csv("ensabmle.csv")

In [117]:
output1=output.mode(axis=1)

In [198]:
output2=pd.concat([output1,test_pred_xgb,test_pred_mlp_tfidf,test_pred_NB,test_pred_lstm],axis=1).mode(axis=1)

In [247]:
final_out=pd.DataFrame(pd.concat([test_tweets.id,output2], axis=1))

In [279]:
pd.DataFrame(test_pred_xgb).to_csv("test.csv")

In [244]:
final_out.to_csv("submission.csv", index=False)